In [1]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general
import scqubits as sc


In [2]:
#comparing H calculations

EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EC = torch.tensor(2.5, requires_grad=True, dtype=torch.double)
EL = torch.tensor(1.9, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
dim = 110

fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'auto_H')
fluxonium2 = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

general.t2(fluxonium2, fluxonium2.t1_supported_noise_channels(), fluxonium2.tphi_supported_noise_channels()).item()-general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels()).item()


 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/general.py: 67

-3.6152079701423645e-05

In [3]:
#compare my pytorch computations to scqubits

fluxonium_sc = sc.Fluxonium(
            EJ = EJ.item(),
            EC = EC.item(),
            EL = EL.item(),
            flux = flux.item(),
           cutoff = dim)

print(fluxonium_sc.t1_quasiparticle_tunneling() - 1/general.effective_t1_rate(fluxonium2 , "t1_quasiparticle_tunneling").item())
print(fluxonium_sc.t1_charge_impedance() - 1/general.effective_t1_rate(fluxonium2 , "t1_charge_impedance").item())
print(fluxonium_sc.t1_capacitive() - 1/general.effective_t1_rate(fluxonium2 , "t1_capacitive").item())
print(fluxonium_sc.t1_inductive() - 1/general.effective_t1_rate(fluxonium2 , "t1_inductive").item())
print(fluxonium_sc.t1_flux_bias_line() - 1/general.effective_t1_rate(fluxonium2 , "t1_flux_bias_line").item())

print(fluxonium_sc.tphi_1_over_f_cc() - 1/general.effective_tphi_rate(fluxonium2 , "tphi_1_over_f_cc").item())
print(fluxonium_sc.tphi_1_over_f_flux() - 1/general.effective_tphi_rate(fluxonium2, "tphi_1_over_f_flux()").item())

See documentation for details.
This warning can be disabled by executing:
scqubits.settings.T1_DEFAULT_WARNING=False

 /Users/judd/Documents/optimisation_of_superconduncting_circuits/venv/lib/python3.8/site-packages/scqubits/core/noise.py: 1195

2.310258610751471e+37
-4.803852372068363
-40847.94101321115
-166662.63067793008
-7.013040117264538e+21
1.792795956134796e-08
7.1810870487846104e+16


In [4]:
#TENSORS

#INPUT

#random values
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

# PARAMETERS
dim = 110
learn_rate = 3e4

# BOUNDS
EJ_bounds = [2.5, 150]
EC_bounds = [1e-3, 8]
EL_bounds = [2e-1, 10]
flux_bounds = [0,1]

# RECORDING VALUES

EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
cost_vals = []

# GRADIENT DESCENT

max_T2 = 0

for i in range(0,700):

    #reinstantiate values
    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EC = torch.tensor(EC.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)
    flux = torch.tensor(flux.item(), requires_grad=True, dtype=torch.double)
    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

    T2 = general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

    if T2 > max_T2:
        max_T2 = T2
    
    #autograd on objective function
    cost = 1/T2
    cost.backward()

    cost_vals.append(cost.item())

    print("CYCLE: ",i)
    print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
    print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())
    print("change EJ,EC,EL,flux = ",EJ.grad.item()*learn_rate,EC.grad.item()*learn_rate,EL.grad.item()*learn_rate,flux.grad.item()*learn_rate)
    
    #update parameters according to gradient
    with torch.no_grad():
        # Near the boundary, we project the gradient onto it. This is the typical approach to constraints.
        if EJ - EJ.grad*learn_rate > EJ_bounds[0] and EJ.data - EJ.grad*learn_rate < EJ_bounds[1]:
            EJ -= EJ.grad*learn_rate
        elif EJ - EJ.grad*learn_rate <= EJ_bounds[0]:
            EJ += (torch.tensor(EJ_bounds[0]) - EJ)
        else:
            EJ += (torch.tensor(EJ_bounds[1]) - EJ)
        EJ_vals.append(EJ.item())
        
        if EC - EC.grad*learn_rate > EC_bounds[0] and EC - EC.grad*learn_rate < EC_bounds[1]:
            EC -= EC.grad*learn_rate
        elif EC - EC.grad*learn_rate <= EC_bounds[0]:
            EC += (torch.tensor(EC_bounds[0]) - EC)
        else:
            EC += (torch.tensor(EC_bounds[1]) - EC)
        EC_vals.append(EC.item())
        
        if EL - EL.grad*learn_rate > EL_bounds[0] and EL - EL.grad*learn_rate < EL_bounds[1]:
            EL -= EL.grad*learn_rate
        elif EL - EL.grad*learn_rate <= EL_bounds[0]:
            EL += (torch.tensor(EL_bounds[0]) - EL)
        else:
            EL += (torch.tensor(EL_bounds[1]) - EL)
        EL_vals.append(EL.item())

    EJ.grad = None
    EC.grad = None
    EL.grad = None
    flux.grad = None

#append last cost value to cost array

fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

T2 = general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

cost = 1/T2
print("CYCLE: END")
print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())

cost_vals.append(cost.item())


CYCLE:  0
T =  547180.3712993208 , cost =  1.8275509364954467e-06 , Best T =  547180.3712993208
EJ,EC,EL,flux =  5.6555810174774495 5.273421204451998 7.057068975192265 0.5
change EJ,EC,EL,flux =  -0.0002992344084575287 0.0013265998923649056 -0.0007514767316623707 224.35943391897402
CYCLE:  1
T =  547204.4717147045 , cost =  1.8274704460408158e-06 , Best T =  547204.4717147045
EJ,EC,EL,flux =  5.655880251885907 5.2720946045596335 7.057820451923927 0.5
change EJ,EC,EL,flux =  -0.00029898453451372714 0.0013273186610124649 -0.0007518718354411372 224.3815940593651


KeyboardInterrupt: 